## ДЗ_7 - Тема “Свёртки”

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше.

### Установим необходимые библиотеки и загрузим даные.

In [1]:
!pip install stop_words pymorphy2

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32896 sha256=a491659231f9530872469f804162a3338263ead1644d1c4ea3e096239d3eca75
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=9b70b09fa6681f6944e69fbd06a69f31a1409c0143decedc6b83f12f8169aeb9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built stop_words docopt


In [2]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 9.8 MB/s eta 0:00:00


In [3]:
!pip install -U gensim

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras import initializers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Зададим гиперпараметры для обучения.**

In [6]:
max_words = 1000
max_len = 80
num_classes = 1

epochs = 20
batch_size = 512
print_batch_n = 100

**Загрузим датасет.**

In [8]:
df =  pd.read_excel('отзывы за лето.xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


**Удалим столбец Date.**

In [9]:
df.drop('Date', axis=1, inplace=True)

**Сделаем предобработку текста.**

In [10]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

**Разделим данные на test и train.**

In [11]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [12]:
df_train.head()

,Rating,Content
3517,5,Наконец-то исправили эту чушь с неоргинальной ...
12399,5,Удобно в использовании
3833,5,Отлично
13250,5,Класс
17941,5,Удобно


In [13]:
df_test.head()

,Rating,Content
1457,5,Всё отлично!!!
19916,5,Мне нравится. Удобно. Еще ни разу не подвел!
2112,3,Иногда слишком долго приходится ждать пока зап...
15154,5,Отличное приложение. Легко отслеживать свои ра...
18329,5,Все грамотно


In [14]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [15]:
df_train.head()

,Rating,Content
3517,5,наконецтый исправить чушь снеоргинальный проши...
12399,5,удобно использование
3833,5,отлично
13250,5,класс
17941,5,удобно


In [16]:
df_test.head()

,Rating,Content
1457,5,отлично
19916,5,нравиться удобно разуна подвести
2112,3,приходиться ждать запуститься приложение бввае...
15154,5,отличный приложение легко отслеживать расход о...
18329,5,грамотно


**Сформируем сорварь. Проиндексируем токены.**

In [17]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [18]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

**Выберем наиболее часто встречающиеся токены.**

In [19]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [20]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
len(vocabulary)

999

In [21]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [22]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [23]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating'])
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [24]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

**Соберем сеть.**

In [25]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=256, input_length=max_len))
model.add(Conv1D(256, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(40))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 256)           256000    
                                                                 
 conv1d (Conv1D)             (None, 78, 256)           196864    
                                                                 
 activation (Activation)     (None, 78, 256)           0         
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 40)                10280     
                                                                 
 activation_1 (Activation)   (None, 40)                0         
                                                        

**Скомпилируем ее.**

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [27]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])
#                     callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 31s 1s/step - loss: 1.1789 - accuracy: 0.6806 - f1_score: 0.1772 - val_loss: 0.9424 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 29s 1s/step - loss: 0.8243 - accuracy: 0.7258 - f1_score: 0.2384 - val_loss: 0.7555 - val_accuracy: 0.7466 - val_f1_score: 0.2776
Epoch 3/20
25/25 [==============================] - 29s 1s/step - loss: 0.6809 - accuracy: 0.7697 - f1_score: 0.2974 - val_loss: 0.7011 - val_accuracy: 0.7502 - val_f1_score: 0.2832
Epoch 4/20
25/25 [==============================] - 29s 1s/step - loss: 0.6251 - accuracy: 0.7810 - f1_score: 0.3178 - val_loss: 0.6857 - val_accuracy: 0.7538 - val_f1_score: 0.3093
Epoch 5/20
25/25 [==============================] - 29s 1s/step - loss: 0.5944 - accuracy: 0.7923 - f1_score: 0.3543 - val_loss: 0.6867 - val_accuracy: 0.7560 - val_f1_score: 0.3234
Epoch 6/20
25/25 [==============================] - 29s 1s/step - loss: 0.5677 - accuracy:

In [28]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 7s 479ms/step - loss: 1.0957 - accuracy: 0.7476 - f1_score: 0.3743


**Загрузим предобученную сеть.**

In [29]:
!wget http://vectors.nlpl.eu/repository/20/220.zip

--2023-10-21 05:59:56--  http://vectors.nlpl.eu/repository/20/220.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638171816 (609M) [application/zip]
Saving to: ‘220.zip’

220.zip             100%[===================>] 608.61M  22.8MB/s    in 28s     

2023-10-21 06:00:24 (22.1 MB/s) - ‘220.zip’ saved [638171816/638171816]



In [30]:
!unzip 220.zip

Archive:  220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [31]:
import gensim

**Загрузим векторы из предобученной модели.**

In [32]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [33]:
len(word_model)

249333

In [34]:
embeddings_index = {}
i = 0
ids = []
with open('model.txt') as f:
    for line in f:
      i += 1
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, "f", sep=" ")
      if i > 1:
        text, sem = word.split('_')
        embeddings_index[text] = coefs
        ids.append(text)


print(f'Добавлено {len(embeddings_index)} векторов.')

Добавлено 205707 векторов.


In [35]:
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

print(f'Добавлены эмбеддинги для {hits} слов ({misses} слов не найдено).')

Добавлены эмбеддинги для 869 слов (130 слов не найдено).


**Подгоним веторы по размеру эмбеддингов из предыдущей модели.**

In [36]:
word_model_matrix = [word_model[i][:256] for i in range(1000)]

In [37]:
initializer = tf.keras.initializers.Constant(word_model_matrix)

model_w = Sequential()
model_w.add(Embedding(input_dim=max_words, output_dim=256, embeddings_initializer =initializer, input_length=max_len))
model_w.add(Conv1D(256, 3))
model_w.add(Activation("relu"))
model_w.add(GlobalMaxPool1D())
model_w.add(Dense(40))
model_w.add(Activation("relu"))
model_w.add(Dense(10))
model_w.add(Activation("relu"))
model_w.add(Dense(num_classes))
model_w.add(Activation('softmax'))
model_w.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 80, 256)           256000    
                                                                 
 conv1d_1 (Conv1D)           (None, 78, 256)           196864    
                                                                 
 activation_4 (Activation)   (None, 78, 256)           0         
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_3 (Dense)             (None, 40)                10280     
                                                                 
 activation_5 (Activation)   (None, 40)                0         
                                                      

In [38]:
model_w.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [39]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')

history_w = model_w.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])
#                     callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 30s 1s/step - loss: 1.0431 - accuracy: 0.7172 - f1_score: 0.2741 - val_loss: 0.8780 - val_accuracy: 0.7220 - val_f1_score: 0.2549
Epoch 2/20
25/25 [==============================] - 35s 1s/step - loss: 0.8447 - accuracy: 0.7429 - f1_score: 0.2783 - val_loss: 0.8294 - val_accuracy: 0.7300 - val_f1_score: 0.2643
Epoch 3/20
25/25 [==============================] - 29s 1s/step - loss: 0.7974 - accuracy: 0.7546 - f1_score: 0.2870 - val_loss: 0.8500 - val_accuracy: 0.7242 - val_f1_score: 0.2528
Epoch 4/20
25/25 [==============================] - 28s 1s/step - loss: 0.7748 - accuracy: 0.7556 - f1_score: 0.2879 - val_loss: 0.7855 - val_accuracy: 0.7350 - val_f1_score: 0.2677
Epoch 5/20
25/25 [==============================] - 29s 1s/step - loss: 0.7252 - accuracy: 0.7639 - f1_score: 0.2984 - val_loss: 0.7783 - val_accuracy: 0.7292 - val_f1_score: 0.2682
Epoch 6/20
25/25 [==============================] - 35s 1s/step - loss: 0.7038 - accuracy:

In [40]:
score = model_w.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 4s 281ms/step - loss: 0.7688 - accuracy: 0.7620 - f1_score: 0.3038


### Вывод: обе модели показали примерно одинаковые результаты по показателю accuracy - 0.7476 у модели которая обучалась сама против 0.7620 у предобученной. Однако показатель f1_score у предобученной модели существенно ниже - 0.3038 против 0.3743. Вероятно, это может быть связано с тем, что не для всех слов в нашем датасете нашлись предобученные эмбеддинги.